In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T


import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 60 # vital length for each NRS
OVERLAP = 2
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수

non_preop_list = []


input_path = '../DL_model/dataset/preprocess4/input3/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])


if True:
    false_row_list_preop = []
    
    preop_ecg_path = "../../cranberry2/Preprocessing/vital_data/ECG_250Hz_preop_2min/"
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_preop_2min/'
    df_preprocess_preop = pickle.load(open('../DL_model/cache/preprocess3/input3/df_preprocess_preop_agender','rb'))
    df_demograph = pd.read_csv('../DL_model/caseids_age_gender.csv')
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    non_list = []
    x_train_preop, y_train_preop = [], []
    x_test_preop, y_test_preop = [], []
    x_val_preop,y_val_preop = [], []
    age_train_preop, gender_train_preop = [], []
    age_test_preop, gender_test_preop = [], []
    age_val_preop, gender_val_preop = [], []    

    cnt = 0
    for _, row in df_preprocess_preop.iterrows():
        cnt += 1
        if cnt>1900:
            break
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_preop)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
        
        
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['Pleth']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, 120*SRATE)
        ppg_per_NRS = np.full(30000, np.nan)
        ppg_per_NRS[0:len(pleth_resamp)] = pleth_resamp
        
        # vital data - ECG (250Hz)       
        try:
            ecg_samp = pickle.load(open(preop_ecg_path+row['file_path'].split(',')[2], 'rb'))
            
        except:
            non_preop_list.append(row['file_path'])
            print('no preop data')
            continue
        else:
            ecg_samp = pd.DataFrame(ecg_samp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()[0:30000]       
            ecg_per_NRS = np.full(30000,np.nan)
            ecg_per_NRS[0:len(ecg_samp)] = ecg_samp
            
            save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preop_'+row['caseids']
            # 한 NRS에 대해 23개의 input 확인
            for i in range(0,30):
                # input이 전처리 통과한 경우
                if row[str(i+1)]:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # input의 normalization
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    pleth_inp = ppg_inp - smooth(ppg_inp)
                    pleth_inp = pleth_inp - np.nanmean(pleth_inp)

                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    ecg_inp = ecg_inp - smooth(ecg_inp)
                    ecg_inp = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)

                    
                    pickle.dump([pleth_inp, ecg_inp], open(save_path+'_{}'.format(i), 'wb'))
                    # 해당 caseid가 test set에 속하는 경우
                    if row['caseids'] in caseid_test:
                        age_test_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_test_preop.append(1)
                        else:
                            gender_test_preop.append(0)
                        x_test_preop.append([pleth_inp, ecg_inp])
                        y_test_preop.append(int(float(row['NRS'])))

                    # 해당 caseid가 train set에 해당하는 경우
                    elif row['caseids'] in caseid_val:
                        age_val_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_val_preop.append(1)
                        else:
                            gender_val_preop.append(0)                    
                        x_val_preop.append([pleth_inp, ecg_inp])
                        y_val_preop.append(int(float(row['NRS'])))

                    elif row['caseids'] in caseid_train:
                        age_train_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_train_preop.append(1)
                        else:
                            gender_train_preop.append(0)                    
                        x_train_preop.append([pleth_inp, ecg_inp])
                        y_train_preop.append(int(float(row['NRS'])))
                        
                    else:
                        non_list.append(row['caseids'])

            print('completed')

    x_train_preop = np.array(x_train_preop, np.float32)
    x_test_preop = np.array(x_test_preop, np.float32)
    y_train_preop = np.array(y_train_preop, int)
    y_test_preop = np.array(y_test_preop, int)
    x_val_preop = np.array(x_val_preop, np.float32)
    y_val_preop = np.array(y_val_preop, int)

    age_train_preop = np.array(age_train_preop, int)
    age_test_preop = np.array(age_test_preop, int)
    age_val_preop = np.array(age_val_preop, int)
    gender_train_preop = np.array(gender_train_preop, int)
    gender_test_preop = np.array(gender_test_preop, int)
    gender_val_preop = np.array(gender_val_preop, int)


    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_preop1.npz', x_train_preop)
    np.savez_compressed(input_path+'x_test_preop1.npz', x_test_preop)
    np.savez_compressed(input_path+'x_val_preop1.npz', x_val_preop)
    np.savez_compressed(input_path+'y_train_preop1.npz', y_train_preop)
    np.savez_compressed(input_path+'y_test_preop1.npz', y_test_preop)
    np.savez_compressed(input_path+'y_val_preop1.npz', y_val_preop)

    np.savez_compressed(input_path+'age_train_preop1.npz', age_train_preop)
    np.savez_compressed(input_path+'age_test_preop1.npz', age_test_preop)
    np.savez_compressed(input_path+'age_val_preop1.npz', age_val_preop)    
    np.savez_compressed(input_path+'gender_train_preop1.npz', gender_train_preop)
    np.savez_compressed(input_path+'gender_test_preop1.npz', gender_test_preop)
    np.savez_compressed(input_path+'gender_val_preop1.npz', gender_val_preop)    

    print('done', flush=True)

    
    
    
print('size of training set(preop):', len(x_train_preop))
print('size of validation set(preop):', len(x_val_preop))
print('size of test set(preop):', len(x_test_preop))

loading data 1/3861 ...completed
loading data 2/3861 ...completed
loading data 3/3861 ...completed
loading data 4/3861 ...completed
loading data 5/3861 ...completed
loading data 6/3861 ...completed
loading data 7/3861 ...completed
loading data 8/3861 ...completed
loading data 9/3861 ...completed
loading data 10/3861 ...completed
loading data 11/3861 ...completed
loading data 12/3861 ...completed
loading data 13/3861 ...completed
loading data 14/3861 ...completed
loading data 15/3861 ...completed
loading data 16/3861 ...completed
loading data 17/3861 ...completed
loading data 18/3861 ...completed
loading data 19/3861 ...completed
loading data 20/3861 ...completed
loading data 21/3861 ...completed
loading data 22/3861 ...completed
loading data 23/3861 ...completed
loading data 24/3861 ...completed
loading data 25/3861 ...completed
loading data 26/3861 ...completed
loading data 27/3861 ...completed
loading data 28/3861 ...completed
loading data 29/3861 ...completed
loading data 30/3861 ..

loading data 473/3861 ...completed
loading data 474/3861 ...completed
loading data 475/3861 ...completed
loading data 476/3861 ...completed
loading data 477/3861 ...completed
loading data 478/3861 ...completed
loading data 479/3861 ...completed
loading data 480/3861 ...completed
loading data 481/3861 ...completed
loading data 482/3861 ...completed
loading data 483/3861 ...completed
loading data 484/3861 ...completed
loading data 485/3861 ...completed
loading data 486/3861 ...completed
loading data 487/3861 ...completed
loading data 488/3861 ...completed
loading data 489/3861 ...completed
loading data 490/3861 ...completed
loading data 491/3861 ...completed
loading data 492/3861 ...completed
loading data 493/3861 ...completed
loading data 494/3861 ...completed
loading data 495/3861 ...completed
loading data 496/3861 ...completed
loading data 497/3861 ...completed
loading data 498/3861 ...completed
loading data 499/3861 ...completed
loading data 500/3861 ...completed
loading data 501/386

loading data 713/3861 ...completed
loading data 714/3861 ...completed
loading data 715/3861 ...completed
loading data 716/3861 ...completed
loading data 717/3861 ...completed
loading data 718/3861 ...completed
loading data 719/3861 ...completed
loading data 720/3861 ...completed
loading data 721/3861 ...completed
loading data 722/3861 ...completed
loading data 723/3861 ...completed
loading data 724/3861 ...completed
loading data 725/3861 ...completed
loading data 726/3861 ...completed
loading data 727/3861 ...completed
loading data 728/3861 ...completed
loading data 729/3861 ...completed
loading data 730/3861 ...completed
loading data 731/3861 ...completed
loading data 732/3861 ...completed
loading data 733/3861 ...completed
loading data 734/3861 ...completed
loading data 735/3861 ...completed
loading data 736/3861 ...completed
loading data 737/3861 ...completed
loading data 738/3861 ...completed
loading data 739/3861 ...completed
loading data 740/3861 ...completed
loading data 741/386

loading data 948/3861 ...completed
loading data 949/3861 ...completed
loading data 950/3861 ...completed
loading data 951/3861 ...completed
loading data 952/3861 ...completed
loading data 953/3861 ...completed
loading data 954/3861 ...completed
loading data 955/3861 ...completed
loading data 956/3861 ...completed
loading data 957/3861 ...completed
loading data 958/3861 ...completed
loading data 959/3861 ...completed
loading data 960/3861 ...completed
loading data 961/3861 ...completed
loading data 962/3861 ...completed
loading data 963/3861 ...completed
loading data 964/3861 ...completed
loading data 965/3861 ...completed
loading data 966/3861 ...completed
loading data 967/3861 ...completed
loading data 968/3861 ...completed
loading data 969/3861 ...completed
loading data 970/3861 ...completed
loading data 971/3861 ...completed
loading data 972/3861 ...completed
loading data 973/3861 ...completed
loading data 974/3861 ...completed
loading data 975/3861 ...completed
loading data 976/386

loading data 1179/3861 ...completed
loading data 1180/3861 ...completed
loading data 1181/3861 ...completed
loading data 1182/3861 ...completed
loading data 1183/3861 ...completed
loading data 1184/3861 ...completed
loading data 1185/3861 ...completed
loading data 1186/3861 ...completed
loading data 1187/3861 ...completed
loading data 1188/3861 ...completed
loading data 1189/3861 ...completed
loading data 1190/3861 ...completed
loading data 1191/3861 ...completed
loading data 1192/3861 ...completed
loading data 1193/3861 ...completed
loading data 1194/3861 ...completed
loading data 1195/3861 ...completed
loading data 1196/3861 ...completed
loading data 1197/3861 ...completed
loading data 1198/3861 ...completed
loading data 1199/3861 ...completed
loading data 1200/3861 ...completed
loading data 1201/3861 ...completed
loading data 1202/3861 ...completed
loading data 1203/3861 ...completed
loading data 1204/3861 ...completed
loading data 1205/3861 ...completed
loading data 1206/3861 ...co

loading data 1634/3861 ...completed
loading data 1635/3861 ...completed
loading data 1636/3861 ...completed
loading data 1637/3861 ...completed
loading data 1638/3861 ...completed
loading data 1639/3861 ...completed
loading data 1640/3861 ...completed
loading data 1641/3861 ...completed
loading data 1642/3861 ...completed
loading data 1643/3861 ...completed
loading data 1644/3861 ...completed
loading data 1645/3861 ...completed
loading data 1646/3861 ...completed
loading data 1647/3861 ...completed
loading data 1648/3861 ...completed
loading data 1649/3861 ...completed
loading data 1650/3861 ...completed
loading data 1651/3861 ...completed
loading data 1652/3861 ...completed
loading data 1653/3861 ...completed
loading data 1654/3861 ...completed
loading data 1655/3861 ...completed
loading data 1656/3861 ...completed
loading data 1657/3861 ...completed
loading data 1658/3861 ...completed
loading data 1659/3861 ...completed
loading data 1660/3861 ...completed
loading data 1661/3861 ...co

# preprocess5/lowess filter

In [2]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 300 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수



input_path = '../DL_model/dataset/preprocess5/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess5/df_preprocess_pacu_agender','rb'))
    df_demograph = pickle.load(open('../DL_model/df_caseids+age+gender', 'rb'))
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    
    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=3000:
            continue
        if cnt>4000:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        LEN = LEN_PER_NRS*SRATE
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['PPG']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, LEN)
        #ppg_per_NRS = np.full(LEN, np.nan)
        #ppg_per_NRS[-len(pleth_resamp):] = pleth_resamp
        ppg_per_NRS = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'], 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        #ecg_per_NRS = np.full(LEN,np.nan)
        #ecg_per_NRS[-len(ecg_samp):] = ecg_samp
        ecg_per_NRS = ecg_samp
        
   
        if len(pleth_resamp) != LEN or len(ecg_samp) != LEN:
            print(row['caseids'])
            break
            

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5/pacu_'+row['file_path']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(23,57):
            # input이 전처리 통과한 경우
            if row[str(i+1)][0]:
                if os.path.exists(save_path+'_{}'.format(i)):
                    ppg_inp, ecg_inp = pickle.load(open(save_path+'_{}'.format(i), 'rb'))
                
                else:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # PPG smoothing
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ppg_inp = ppg_inp - smooth(ppg_inp)
                    
                    # ECG smoothing
                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ecg_inp = ecg_inp - smooth(ecg_inp)                    
                
                    # pickle에 저장
                    pickle.dump([ppg_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    print('{}...'.format(i+1), end='')
        print('done')
'''                
                # normalization
                pleth_inp = ppg_inp - np.nanmean(ppg_inp)
                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp2])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp2])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp2])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')
        
        
    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu2.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu2.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu2.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu2.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu2.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu2.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu2.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu2.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu2.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu2.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu2.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu2.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))
'''

loading data 3001/6963 ...25...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...52...53...54...55...56...57...done
loading data 3002/6963 ...27...28...29...30...31...37...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...done
loading data 3003/6963 ...28...37...45...46...47...48...49...50...done
loading data 3004/6963 ...24...25...28...29...30...31...32...33...34...35...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...done
loading data 3005/6963 ...27...45...47...48...54...55...done
loading data 3006/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...49...50...51...52...53...54...55...56...57...done
loading data 3007/6963 ...24...25...26...27...28...29...30...35...36...37...38...39...40...41...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3008/6963 

loading data 3143/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3144/6963 ...24...25...26...27...28...29...33...57...done
loading data 3145/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3146/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3147/6963 ...24...25...26...27...28...32...33...38...39...40...41...42...43...46...47...48...49...56...57...done
loading data 3148/6963 ...done
loading data 3149/6963 ...24...25...26...34...35...44...45...57...done
loading data 3150/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45

loading data 3268/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3269/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...done
loading data 3270/6963 ...24...25...26...30...31...32...33...34...35...36...37...38...39...40...41...42...43...51...52...53...54...55...56...57...done
loading data 3271/6963 ...24...25...26...27...28...29...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...54...55...56...57...done
loading data 3272/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3273/6963 ...24...25...26...27...28...33...36...37...38...39...40...41...42...48...49...50...51..

loading data 3402/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3403/6963 ...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...done
loading data 3404/6963 ...24...25...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3405/6963 ...27...28...57...done
loading data 3406/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...54...55...56...57...done
loading data 3407/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3408/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3409/6963 ...32...33

loading data 3538/6963 ...28...29...30...31...32...33...34...35...36...38...39...40...41...42...43...44...50...51...52...done
loading data 3539/6963 ...24...25...26...27...28...29...30...31...32...33...34...36...37...38...39...40...41...42...43...44...45...46...47...48...50...51...52...53...done
loading data 3540/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...46...47...48...done
loading data 3541/6963 ...25...26...35...36...37...38...39...40...41...42...48...52...54...done
loading data 3542/6963 ...24...25...26...27...28...29...30...35...36...37...38...39...40...41...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3543/6963 ...29...30...31...done
loading data 3544/6963 ...26...27...28...29...30...31...32...45...46...47...48...49...50...51...52...53...done
loading data 3545/6963 ...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...49...50...51...done
lo

loading data 3667/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...50...51...52...53...54...55...56...57...done
loading data 3668/6963 ...29...30...31...32...37...39...48...49...50...51...52...57...done
loading data 3669/6963 ...24...25...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3670/6963 ...26...27...28...38...40...45...done
loading data 3671/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...52...53...54...55...56...57...done
loading data 3672/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...done
loading data 3673/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42.

loading data 3802/6963 ...25...26...27...28...29...30...31...32...done
loading data 3803/6963 ...24...25...27...28...29...30...31...32...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3804/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3805/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...39...40...41...42...43...47...48...49...50...51...done
loading data 3806/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...47...48...49...50...56...57...done
loading data 3807/6963 ...24...25...26...27...28...29...35...38...43...44...49...50...51...52...53...done
loading data 3808/6963 ...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48.

loading data 3938/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...52...53...54...55...56...57...done
loading data 3939/6963 ...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...46...47...48...49...55...56...57...done
loading data 3940/6963 ...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3941/6963 ...24...25...26...27...28...29...30...31...32...34...35...36...49...50...51...52...53...54...55...56...57...done
loading data 3942/6963 ...24...25...26...27...28...done
loading data 3943/6963 ...54...55...56...57...done
loading data 3944/6963 ...24...25...26...27...28...29...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...done
loading data 3945/6963 ...35...36...37...38...39...40...done
loading data 3946/6963 ...24...2

"                \n                # normalization\n                pleth_inp = ppg_inp - np.nanmean(ppg_inp)\n                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)\n                \n                \n                # 해당 caseid가 test set에 속하는 경우\n                if row['caseids'] in caseid_test:\n                    age_test_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_test_pacu.append(1)\n                    else:\n                        gender_test_pacu.append(0)\n                    x_test_pacu.append([pleth_inp, ecg_inp2])\n                    y_test_pacu.append(int(float(row['NRS'])))\n\n                # 해당 caseid가 train set에 해당하는 경우\n                elif row['caseids'] in caseid_val:\n                    age_val_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_val_pacu.append(1)\n                    else:\n          

In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 300 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수


vital_list = pickle.load(open('../Preprocessing/201227_modified_vital','rb'))
input_path = '../DL_model/dataset/preprocess5/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess5/df_preprocess_pacu_agender','rb'))
    df_demograph = pickle.load(open('../DL_model/df_caseids+age+gender', 'rb'))
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    
    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=3000:
            continue
        if cnt>4000:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        LEN = LEN_PER_NRS*SRATE
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['PPG']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, LEN)
        #ppg_per_NRS = np.full(LEN, np.nan)
        #ppg_per_NRS[-len(pleth_resamp):] = pleth_resamp
        ppg_per_NRS = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'], 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        #ecg_per_NRS = np.full(LEN,np.nan)
        #ecg_per_NRS[-len(ecg_samp):] = ecg_samp
        ecg_per_NRS = ecg_samp
        
   
        if len(pleth_resamp) != LEN or len(ecg_samp) != LEN:
            print(row['caseids'])
            break
            

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5/pacu_'+row['file_path']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(23,57):
            # input이 전처리 통과한 경우
            if row[str(i+1)][0]:
                if os.path.exists(save_path+'_{}'.format(i)) and row['file_path'] not in vital_list:
                    ppg_inp, ecg_inp = pickle.load(open(save_path+'_{}'.format(i), 'rb'))
                
                else:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # PPG smoothing
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ppg_inp = ppg_inp - smooth(ppg_inp)
                    
                    # ECG smoothing
                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ecg_inp = ecg_inp - smooth(ecg_inp)                    
                
                    # pickle에 저장
                    pickle.dump([ppg_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    print('{}...'.format(i+1), end='')
        print('done')
'''                
                # normalization
                pleth_inp = ppg_inp - np.nanmean(ppg_inp)
                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp2])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp2])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp2])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')
        
        
    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu2.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu2.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu2.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu2.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu2.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu2.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu2.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu2.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu2.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu2.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu2.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu2.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))
'''

loading data 3001/9949 ...done
loading data 3002/9949 ...done
loading data 3003/9949 ...24...25...done
loading data 3004/9949 ...done
loading data 3005/9949 ...done
loading data 3006/9949 ...done
loading data 3007/9949 ...done
loading data 3008/9949 ...done
loading data 3009/9949 ...done
loading data 3010/9949 ...done
loading data 3011/9949 ...done
loading data 3012/9949 ...done
loading data 3013/9949 ...done
loading data 3014/9949 ...done
loading data 3015/9949 ...done
loading data 3016/9949 ...27...28...29...30...31...40...41...42...done
loading data 3017/9949 ...done
loading data 3018/9949 ...done
loading data 3019/9949 ...done
loading data 3020/9949 ...done
loading data 3021/9949 ...done
loading data 3022/9949 ...done
loading data 3023/9949 ...done
loading data 3024/9949 ...done
loading data 3025/9949 ...done
loading data 3026/9949 ...done
loading data 3027/9949 ...done
loading data 3028/9949 ...done
loading data 3029/9949 ...done
loading data 3030/9949 ...done
loading data 3031/99

loading data 3260/9949 ...done
loading data 3261/9949 ...done
loading data 3262/9949 ...done
loading data 3263/9949 ...done
loading data 3264/9949 ...done
loading data 3265/9949 ...done
loading data 3266/9949 ...done
loading data 3267/9949 ...done
loading data 3268/9949 ...done
loading data 3269/9949 ...done
loading data 3270/9949 ...done
loading data 3271/9949 ...done
loading data 3272/9949 ...done
loading data 3273/9949 ...done
loading data 3274/9949 ...done
loading data 3275/9949 ...done
loading data 3276/9949 ...done
loading data 3277/9949 ...done
loading data 3278/9949 ...done
loading data 3279/9949 ...done
loading data 3280/9949 ...done
loading data 3281/9949 ...done
loading data 3282/9949 ...done
loading data 3283/9949 ...done
loading data 3284/9949 ...done
loading data 3285/9949 ...done
loading data 3286/9949 ...done
loading data 3287/9949 ...done
loading data 3288/9949 ...done
loading data 3289/9949 ...done
loading data 3290/9949 ...done
loading data 3291/9949 ...done
loading 

loading data 3524/9949 ...done
loading data 3525/9949 ...done
loading data 3526/9949 ...done
loading data 3527/9949 ...done
loading data 3528/9949 ...done
loading data 3529/9949 ...done
loading data 3530/9949 ...done
loading data 3531/9949 ...done
loading data 3532/9949 ...done
loading data 3533/9949 ...done
loading data 3534/9949 ...done
loading data 3535/9949 ...done
loading data 3536/9949 ...done
loading data 3537/9949 ...done
loading data 3538/9949 ...done
loading data 3539/9949 ...done
loading data 3540/9949 ...done
loading data 3541/9949 ...done
loading data 3542/9949 ...done
loading data 3543/9949 ...done
loading data 3544/9949 ...done
loading data 3545/9949 ...done
loading data 3546/9949 ...done
loading data 3547/9949 ...done
loading data 3548/9949 ...done
loading data 3549/9949 ...done
loading data 3550/9949 ...done
loading data 3551/9949 ...38...done
loading data 3552/9949 ...done
loading data 3553/9949 ...41...42...43...44...45...46...done
loading data 3554/9949 ...done
load

loading data 3778/9949 ...done
loading data 3779/9949 ...done
loading data 3780/9949 ...done
loading data 3781/9949 ...done
loading data 3782/9949 ...done
loading data 3783/9949 ...done
loading data 3784/9949 ...done
loading data 3785/9949 ...done
loading data 3786/9949 ...done
loading data 3787/9949 ...33...done
loading data 3788/9949 ...done
loading data 3789/9949 ...done
loading data 3790/9949 ...done
loading data 3791/9949 ...done
loading data 3792/9949 ...done
loading data 3793/9949 ...done
loading data 3794/9949 ...done
loading data 3795/9949 ...done
loading data 3796/9949 ...done
loading data 3797/9949 ...done
loading data 3798/9949 ...done
loading data 3799/9949 ...done
loading data 3800/9949 ...done
loading data 3801/9949 ...done
loading data 3802/9949 ...done
loading data 3803/9949 ...done
loading data 3804/9949 ...done
loading data 3805/9949 ...done
loading data 3806/9949 ...done
loading data 3807/9949 ...done
loading data 3808/9949 ...done
loading data 3809/9949 ...done
loa

"                \n                # normalization\n                pleth_inp = ppg_inp - np.nanmean(ppg_inp)\n                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)\n                \n                \n                # 해당 caseid가 test set에 속하는 경우\n                if row['caseids'] in caseid_test:\n                    age_test_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_test_pacu.append(1)\n                    else:\n                        gender_test_pacu.append(0)\n                    x_test_pacu.append([pleth_inp, ecg_inp2])\n                    y_test_pacu.append(int(float(row['NRS'])))\n\n                # 해당 caseid가 train set에 해당하는 경우\n                elif row['caseids'] in caseid_val:\n                    age_val_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_val_pacu.append(1)\n                    else:\n          